In [2]:
import numpy as np
import pandas as pd
import sys
import os

# Add src to path for importing preprocessing module
sys.path.insert(0, os.path.abspath('..'))

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from preprocessing.feature_pipeline import build_default_pipeline
from xgboost import XGBRegressor


import joblib

In [3]:
import sys
sys.executable

'd:\\house_price\\house-price-prediction-ml\\venv\\Scripts\\python.exe'

In [4]:
import pandas as pd

df = pd.read_excel("../../data/raw/Case Study 1 Data.xlsx")
df.head()

,Property ID,Location,Size,Bedrooms,Bathrooms,Year Built,Condition,Type,Date Sold,Price
0,SI_000001,CityA,3974.0,2.0,2.0,2007.0,Good,Single Family,2020-11-02,324000.0
1,SI_000002,CityA,1660.0,2.0,3.0,1934.0,Good,Single Family,2022-10-23,795000.0
2,SI_000003,CityC,2094.0,2.0,2.0,1950.0,Good,Single Family,2020-11-30,385000.0
3,SI_000004,CityB,1930.0,2.0,3.0,1905.0,Good,Single Family,2021-12-09,651000.0
4,SI_000005,CityB,1895.0,5.0,2.0,1936.0,New,Single Family,2024-10-30,1878000.0


In [5]:
X = df.drop(columns=["Price"])
y = df["Price"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [6]:
# First, fit the feature pipeline on training data
feature_pipeline = build_default_pipeline()
feature_pipeline.fit(X_train)

# Transform training and test data
X_train_transformed = feature_pipeline.transform(X_train)
X_test_transformed = feature_pipeline.transform(X_test)

# Convert to numpy arrays for sklearn models
X_train_array = X_train_transformed.values
X_test_array = X_test_transformed.values

# Now define models (without the feature pipeline step)
baseline_models = {
    "Linear": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "RandomForest": RandomForestRegressor(random_state=42),
    "GradientBoosting": GradientBoostingRegressor(random_state=42),

    "XGBoost": XGBRegressor(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        random_state=42,
        n_jobs=-1
    )
}


print("Feature pipeline fitted. Transformed shape:")
print(f"  X_train_transformed: {X_train_array.shape}")
print(f"  X_test_transformed: {X_test_array.shape}")

Feature pipeline fitted. Transformed shape:
  X_train_transformed: (197737, 10)
  X_test_transformed: (49435, 10)


In [7]:
baseline_results = []

# Remove NaN values from y_test
valid_idx = ~y_test.isna()
y_test_clean = y_test[valid_idx].reset_index(drop=True)
X_test_array_clean = X_test_array[valid_idx.values]

n = X_test_array_clean.shape[0]  # number of test samples
p = X_test_array_clean.shape[1]  # number of features after preprocessing

for name, model in baseline_models.items():
    # Train model
    model.fit(X_train_array, y_train)
    preds = model.predict(X_test_array_clean)

    rmse = np.sqrt(mean_squared_error(y_test_clean, preds))
    r2 = r2_score(y_test_clean, preds)

    # Calculate adjusted R2
    adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)

    baseline_results.append({
        "Model": name,
        "RMSE": rmse,
        "R2": r2,
        "Adjusted_R2": adjusted_r2
    })

baseline_df = pd.DataFrame(baseline_results).sort_values("RMSE")
print(baseline_df)


              Model           RMSE        R2  Adjusted_R2
5           XGBoost  143286.911865  0.619818     0.619731
4  GradientBoosting  143880.634328  0.616660     0.616573
2             Lasso  151304.745463  0.576080     0.575983
1             Ridge  151304.797158  0.576079     0.575983
0            Linear  151304.829048  0.576079     0.575983
3      RandomForest  151822.349698  0.573174     0.573077


In [8]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
xgb_model = XGBRegressor(
    objective="reg:squarederror",
    random_state=42,
    n_jobs=1
)


In [9]:
xgb_param_dist = {
    "n_estimators": [300, 500, 800],
    "max_depth": [3, 5, 7, 9],
    "learning_rate": [0.01, 0.03, 0.05, 0.1],
    "subsample": [0.6, 0.8, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "reg_alpha": [0, 0.1, 1],
    "reg_lambda": [1, 5, 10]
}


In [10]:
xgb_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=xgb_param_dist,
    n_iter=20,
    cv=3,
    scoring="neg_root_mean_squared_error",
    verbose=1,
    n_jobs=1,
    random_state=42
)

xgb_search.fit(X_train_array, y_train)


Fitting 3 folds for each of 20 candidates, totalling 60 fits


,"estimator estimator: estimator objectAn object of that type is instantiated for each grid point.This is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.","XGBRegressor(...ree=None, ...)"
,"param_distributions param_distributions: dict or list of dictsDictionary with parameters names (`str`) as keys and distributionsor lists of parameters to try. Distributions must provide a ``rvs``method for sampling (such as those from scipy.stats.distributions).If a list is given, it is sampled uniformly.If a list of dicts is given, first a dict is sampled uniformly, andthen a parameter is sampled using that dict as above.","{'colsample_bytree': [0.6, 0.8, ...], 'learning_rate': [0.01, 0.03, ...], 'max_depth': [3, 5, ...], 'n_estimators': [300, 500, ...], ...}"
,"n_iter n_iter: int, default=10Number of parameter settings that are sampled. n_iter tradesoff runtime vs quality of the solution.",20
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.If None, the estimator's score method is used.",'neg_root_mean_squared_error'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given the ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``RandomizedSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strate

In [11]:
print("Best XGBoost RMSE (CV):", -xgb_search.best_score_)
print("Best XGBoost Params:", xgb_search.best_params_)

# Compute tuned XGBoost metrics on the cleaned test set
best_xgb = xgb_search.best_estimator_
preds = best_xgb.predict(X_test_array_clean)
rmse = np.sqrt(mean_squared_error(y_test_clean, preds))
r2 = r2_score(y_test_clean, preds)
n = X_test_array_clean.shape[0]
p = X_test_array_clean.shape[1]
adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
print("Tuned XGBoost Test RMSE:", rmse)
print("Tuned XGBoost Test R2:", r2)
print("Tuned XGBoost Test Adjusted R2:", adjusted_r2)


Best XGBoost RMSE (CV): 135908.62806826446
Best XGBoost Params: {'subsample': 0.8, 'reg_lambda': 10, 'reg_alpha': 0, 'n_estimators': 800, 'max_depth': 5, 'learning_rate': 0.03, 'colsample_bytree': 0.6}
Tuned XGBoost Test RMSE: 143188.9464832741
Tuned XGBoost Test R2: 0.6203372040442248
Tuned XGBoost Test Adjusted R2: 0.6202508915437233


In [17]:
# Save the final tuned XGBoost model
model_path = "../../models/xgboost_house_price_model.pkl"
joblib.dump(best_xgb, model_path)
print(f"Model saved successfully at: {model_path}")

Model saved successfully at: ../../models/xgboost_house_price_model.pkl


In [18]:
# Save the feature pipeline for preprocessing unseen data
pipeline_path = "../../models/feature_pipeline.pkl"
joblib.dump(feature_pipeline, pipeline_path)
print(f"Feature pipeline saved successfully at: {pipeline_path}")

Feature pipeline saved successfully at: ../../models/feature_pipeline.pkl


In [19]:
import pandas as pd

new_house = pd.DataFrame([{
    "Size": 2000,
    "Bedrooms": 3,
    "Bathrooms": 2,
    "Year Built": 2018,
    "Condition": "Good",
    "Type": "Apartment",
    "Location": "Delhi",
    "Date Sold": "2026-02-01"
}])


In [20]:
X_new = feature_pipeline.transform(new_house)


In [21]:
predicted_price = model.predict(X_new.values)
print("Predicted Price:", predicted_price[0])


Predicted Price: 410761.94


In [23]:
bundle = {
    "feature_pipeline": joblib.load("../../models/feature_pipeline.pkl"),
    "model": joblib.load("../../models/xgboost_house_price_model.pkl")
}

joblib.dump(bundle, "../../models/house_price_model_bundle.pkl")

['../../models/house_price_model_bundle.pkl']